In [1]:
#Author: Sang Do 2024
#predict loan status https://www.kaggle.com/datasets/ethon0426/lending-club-20072020q1

In [2]:
import pandas as pd
import numpy as np

import time
import calendar

In [3]:
def getCurrentTimestamp():
    return int(calendar.timegm(time.gmtime()))

In [4]:
start_time = getCurrentTimestamp()
path = 'Loan_status_2007-2020Q3.csv' #1.7GB
print('Begin reading CSV file, please wait ...')
df = pd.read_csv(path,low_memory=False)
print('Complete reading CSV file')
total_time = getCurrentTimestamp() - start_time
print(total_time) #in seconds (approx 3.5 mins)
#df -> 2925493 rows × 142 columns
df.describe()

Begin reading CSV file, please wait ...
Complete reading CSV file
232


,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
count,2.925493e+06,2.925492e+06,2.925492e+06,2.925492e+06,2.925492e+06,2.925488e+06,2.922384e+06,2.925463e+06,2.925492e+06,2.925492e+06,...,197824.000000,197824.000000,197824.000000,143638.000000,182076.000000,143638.000000,143637.000000,179240.000000,182076.000000,182076.000000
mean,8.697419e+04,1.535878e+04,1.535470e+04,1.534005e+04,4.523914e+02,7.993727e+04,1.929648e+01,2.897012e-01,7.005110e+02,7.045112e+02,...,12.544398,0.038327,0.067545,2.233427,119.887400,2.233427,2.323148,369.367460,11993.128927,745.504875
std,8.048842e+04,9.478383e+03,9.477145e+03,9.480634e+03,2.729478e+02,1.117476e+05,1.571977e+01,8.436819e-01,3.376565e+01,3.376653e+01,...,8.206977,0.362390,0.371836,0.565689,247.522976,0.565689,6.179453,468.696526,9162.902724,2095.316599
min,0.000000e+00,5.000000e+02,5.000000e+02,0.000000e+00,4.930000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,6.100000e+02,6.140000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.284100e+04,8.000000e+03,8.000000e+03,8.000000e+03,2.535000e+02,4.700000e+04,1.208000e+01,0.000000e+00,6.750000e+02,6.790000e+02,...,7.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,104.280000,4749.142500,259.040000
50%,6.892500e+04,1.300000e+04,1.300000e+04,1.300000e+04,3.823000e+02,6.600000e+04,1.810000e+01,0.000000e+00,6.950000e+02,6.990000e+02,...,11.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,249.474420,9967.160000,423.200000
75%,1.072940e+05,2.000000e+04,2.000000e+04,2.000000e+04,6.039200e+02,9.500000e+04,2.488000e+01,0.000000e+00,7.200000e+02,7.240000e+02,...,17.000000,0.000000,0.000000,2.000000,101.940000,2.000000,0.000000,482.532500,17640.670000,679.580000
max,4.210940e+05,4.000000e+04,4.000000e+04,4.000000e+04,1.719830e+03,1.100000e+08,9.990000e+02,5.800000e+01,8.450000e+02,8.500000e+02,...,121.000000,21.000000,23.000000,4.000000,2797.500000,4.000000,82.000000,11581.820000,42135.550000,40270.980000


In [5]:
#show random rows to ensure data loaded
df.sample(n=5)

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
2362855,73999,71836269,7000.0,7000.0,6950.0,36 months,9.16%,223.12,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1994866,127105,60892425,21000.0,21000.0,21000.0,60 months,14.65%,495.74,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2128976,261215,51165314,4000.0,4000.0,4000.0,36 months,15.61%,139.86,D,D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2856388,36346,108234342,6600.0,6600.0,6600.0,36 months,15.05%,228.96,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1187582,104599,150163843,10000.0,10000.0,10000.0,60 months,14.74%,236.54,C,C2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [6]:
#1. Clean data process

In [7]:
#1a. Select only rows that contain the “loan_status”: “Fully Paid” or “Charged Off”

In [8]:
df = df[(df['loan_status'] == 'Fully Paid') | (df['loan_status'] == 'Charged Off')]
df['loan_status'].value_counts()

loan_status
Fully Paid     1497783
Charged Off     362548
Name: count, dtype: int64

In [9]:
#1b. Replace “Fully Paid” as 0, “Charged Off” as 1 in the column “loan_status”

In [10]:
df = df.replace({'loan_status':'Fully Paid'}, 0)
df = df.replace({'loan_status':'Charged Off'}, 1)

In [11]:
df['loan_status'].value_counts()

loan_status
0    1497783
1     362548
Name: count, dtype: int64

In [12]:
#1c. Drop duplicated rows

In [13]:
df = df.drop_duplicates()
len(df)

1860331

In [14]:
#1d. Drop columns with all null, same, or unique values: This kind of column has no determination for the result.

In [15]:
total_rows = len(df)
print (total_rows)
column_names = []

for col in df.columns:
    if df[col].isnull().sum() == total_rows or df[col].isna().sum() == total_rows:
    	#column has all empty values
        print('column name :' + col)
        print(df[col].isnull().sum())
        column_names.append(col)
    if df[col].nunique() == total_rows or df[col].nunique() == 1:
    	#column has unique key or only single 1 value
        print('column name 1 :' + col)
        print(df[col].nunique())
        column_names.append(col)
print(column_names)

1860331
column name 1 :id
1860331
column name 1 :pymnt_plan
1
column name 1 :url
1860331
column name 1 :out_prncp
1
column name 1 :out_prncp_inv
1
column name :next_pymnt_d
1860331
column name 1 :policy_code
1
['id', 'pymnt_plan', 'url', 'out_prncp', 'out_prncp_inv', 'next_pymnt_d', 'policy_code']


In [16]:
#drop those columns
df = df.drop(columns = column_names)
df.describe()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
count,1.860331e+06,1.860331e+06,1.860331e+06,1.860331e+06,1.860331e+06,1.860331e+06,1.859223e+06,1.860331e+06,1.860331e+06,1.860331e+06,...,58632.000000,58632.000000,58632.000000,12695.000000,22049.000000,12695.000000,12695.000000,19213.000000,22049.000000,22049.000000
mean,1.022656e+05,1.458827e+04,1.458218e+04,1.456467e+04,4.402559e+02,7.736956e+04,1.856774e+01,3.128750e-01,6.976785e+02,7.016786e+02,...,12.580639,0.047176,0.080144,2.853013,289.396675,2.853013,12.278220,175.133475,5763.524875,2232.937401
std,9.298534e+04,8.970471e+03,8.967823e+03,8.969319e+03,2.669094e+02,1.178218e+05,1.309153e+01,8.754565e-01,3.270865e+01,3.270940e+01,...,8.217732,0.415817,0.413078,0.357208,285.567423,0.357208,9.984442,307.335793,7910.200323,5231.597305
min,0.000000e+00,5.000000e+02,5.000000e+02,0.000000e+00,4.930000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,6.250000e+02,6.290000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.732400e+04,7.950000e+03,7.925000e+03,7.800000e+03,2.469900e+02,4.600000e+04,1.182000e+01,0.000000e+00,6.700000e+02,6.740000e+02,...,7.000000,0.000000,0.000000,3.000000,71.840000,3.000000,0.000000,0.000000,0.000000,100.000000
50%,7.675100e+04,1.200000e+04,1.200000e+04,1.200000e+04,3.736300e+02,6.500000e+04,1.771000e+01,0.000000e+00,6.900000e+02,6.940000e+02,...,11.000000,0.000000,0.000000,3.000000,203.850000,3.000000,13.000000,0.000000,1368.410000,316.510000
75%,1.272705e+05,2.000000e+04,2.000000e+04,2.000000e+04,5.849600e+02,9.200000e+04,2.429000e+01,0.000000e+00,7.150000e+02,7.190000e+02,...,17.000000,0.000000,0.000000,3.000000,410.220000,3.000000,21.000000,239.760000,9659.450000,929.200000
max,4.210940e+05,4.000000e+04,4.000000e+04,4.000000e+04,1.719830e+03,1.100000e+08,9.990000e+02,4.200000e+01,8.450000e+02,8.500000e+02,...,97.000000,20.000000,16.000000,4.000000,2797.500000,4.000000,38.000000,3466.900000,40306.410000,40270.980000


In [17]:
df = df.drop(columns = ['Unnamed: 0'])

In [18]:
#1e. Remove the text "months" in feature "term"

In [19]:
df['term'] = df['term'].replace('months', '', regex=True)

In [20]:
df['term']

0           36 
1           60 
2           36 
3           36 
4           60 
           ... 
2925488     60 
2925489     36 
2925490     36 
2925491     36 
2925492     60 
Name: term, Length: 1860331, dtype: object

In [21]:
#1e. Remove the text "%" in feature "int_rate" and "revol_util"

In [22]:
df['int_rate'] = df['int_rate'].replace('%', '', regex=True)
df['revol_util'] = df['revol_util'].replace('%', '', regex=True)
df['int_rate']
df['revol_util']

0          83.7
1           9.4
2          98.5
3            21
4          53.9
           ... 
2925488    24.9
2925489    15.7
2925490      47
2925491    10.1
2925492    72.6
Name: revol_util, Length: 1860331, dtype: object

In [23]:
df['int_rate']

0           10.65
1           15.27
2           15.96
3           13.49
4           12.69
            ...  
2925488     23.99
2925489      7.99
2925490     16.99
2925491     11.44
2925492     25.49
Name: int_rate, Length: 1860331, dtype: object

In [24]:
#1g.	Transform text into numeric values

In [25]:
def potentially_numeric(col):
  # Try converting to numeric (ignore errors)
  try:
    pd.to_numeric(col, errors="raise")
    return True
  except:
    return False

# Use list comprehension and filter by potentially numeric columns
potentially_numeric_cols = [col for col in df.columns if potentially_numeric(df[col])]

print(potentially_numeric_cols)

#find columns contain strings
string_cols = (df.dtypes == 'object') 
non_number_cols = []
for col in string_cols.index.tolist():
    if col in potentially_numeric_cols:
        #this col is numberic type
        continue;
    non_number_cols.append(col)
non_number_cols

['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'annual_inc', 'loan_status', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_ac

['grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'debt_settlement_flag']

In [26]:
non_number_cols

['grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'verification_status_joint',
 'sec_app_earliest_cr_line',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_loan_status',
 'debt_settlement_flag']

In [27]:
#convert string values to number values
for col in non_number_cols:
    cat, new_col = pd.factorize(df[col])
    df[col] = cat

In [28]:
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,5000.0,5000.0,4975.0,36,10.65,162.87,0,0,-1,0,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
1,2500.0,2500.0,2500.0,60,15.27,59.83,1,1,0,1,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
2,2400.0,2400.0,2400.0,36,15.96,84.33,1,2,-1,0,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
3,10000.0,10000.0,10000.0,36,13.49,339.31,1,3,1,0,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
4,3000.0,3000.0,3000.0,60,12.69,67.79,0,4,2,2,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925488,24000.0,24000.0,24000.0,60,23.99,690.30,3,29,163799,1,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
2925489,10000.0,10000.0,10000.0,36,7.99,313.32,2,17,30782,0,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
2925490,10050.0,10050.0,10050.0,36,16.99,358.26,5,10,29464,4,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0
2925491,6000.0,6000.0,6000.0,36,11.44,197.69,0,13,456675,7,...,-1,-1,-1,NaN,NaN,-1,NaN,NaN,NaN,0


In [29]:
df['term'].value_counts()

term
36     1393299
60      467032
Name: count, dtype: int64

In [30]:
pd.set_option('future.no_silent_downcasting', True) #to avoid Downcasting behavior
df['term'] = df['term'].replace('36', 0, regex=True)

In [31]:
df['term'] = df['term'].replace('60', 1, regex=True)

In [32]:
#fill empty values by mean -> taking much time

In [33]:
print('Begin filling empty cell by mean of its column')
start_time = getCurrentTimestamp()
for col in df.columns:
    try:
        df[col] = df[col].fillna(df[col].mean())
    except:
        print('having error in col: ' + col)

total_time = getCurrentTimestamp() - start_time
print(total_time) #in seconds

Begin filling empty cell by mean of its column
having error in col: int_rate
having error in col: revol_util
461


In [48]:
#convert string to number for columns having error
try:
    df['int_rate'] = pd.to_numeric(df['int_rate'])
    df['revol_util'] = pd.to_numeric(df['revol_util'])
except:
    print('do nothing')

In [49]:
#fill null data by mean again
df['int_rate'] = df['int_rate'].fillna(df['int_rate'].mean())
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].mean())
df['revol_util']

0          83.7
1           9.4
2          98.5
3          21.0
4          53.9
           ... 
2925488    24.9
2925489    15.7
2925490    47.0
2925491    10.1
2925492    72.6
Name: revol_util, Length: 1860331, dtype: float64

In [59]:
X = df.drop(['loan_status'], axis=1) #input
Y = df['loan_status'] #output
Y = Y.astype('int') #to make sure output is binary

In [60]:
X.shape,Y.shape 

((1860331, 133), (1860331,))

In [61]:
#split into training set & test set (default configs)
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    test_size=0.2, #choose 80% of data to be trained
                                                    random_state=1) 
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1488264, 133), (372067, 133), (1488264,), (372067,))

In [62]:
#train and predict -> taking much time

In [63]:
#random forest
from sklearn.ensemble import RandomForestClassifier

start_time = getCurrentTimestamp()
print('Begin training RDF, please wait ...')
model_rdf = RandomForestClassifier(n_estimators=100, random_state=42)  #default parameters
model_rdf.fit(X_train, Y_train)
print('Finish training RDF ...')

total_time = getCurrentTimestamp() - start_time
print(total_time) #in seconds

#predict by our model
y_pred = model_rdf.predict(X_test)

Begin training RDF, please wait ...
Finish training RDF ...
598


In [ ]:
#print more metrics

In [64]:
from sklearn import metrics

accuracy = 100*metrics.accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 99.77504051689615


In [65]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    299577
           1       1.00      0.99      0.99     72490

    accuracy                           1.00    372067
   macro avg       1.00      0.99      1.00    372067
weighted avg       1.00      1.00      1.00    372067



In [ ]:
#export model to file

In [ ]:
#import pickle

#with open('rdf_model.pkl', 'wb') as f:
  #pickle.dump(model_rdf, f)

In [ ]:
#export some rows to file for testing docker
#X.sample(2).to_csv('loan_predict_test.csv')

In [ ]:
#Reduce size of data for training SVM because it is taking too much time for original dataset

In [66]:
from sklearn.model_selection import StratifiedShuffleSplit

# Define the target variable
# target_variable = 'loan_status'
# # splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2)  # Adjust test size as needed
# splitter = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=0) #reduce 50% in size
# #splitter=StratifiedShuffleSplit(n_splits=1,random_state=12) #we can make a number of combinations of split
# for train,test in splitter.split(X,Y):     #this will splits the index
#     X_train_SS = X.iloc[train]
#     Y_train_SS = Y.iloc[train]
#     X_test_SS = X.iloc[test]
#     Y_test_SS = Y.iloc[test]
# print(Y_train_SS.value_counts())  
# print(Y_test_SS.value_counts())
#Size is down 50%

In [ ]:
#Try SVM
# from sklearn.svm import SVC

# svm_model = SVC(kernel='linear') #default
# start_time = getCurrentTimestamp()
# print('Begin training SVM, please wait ...')
# svm_model.fit(X_train_SS, Y_train_SS)
# print('Finish training SVM ...')
# total_time = getCurrentTimestamp() - start_time
# print(total_time) #in seconds
# y_pred_svm = svm_model.predict(X_test_SS) #predict by our model

In [67]:
#Try LinearSVC
from sklearn.svm import LinearSVC

start_time = getCurrentTimestamp()
print('Begin training LinearSVC, please wait ...')
linear_model = LinearSVC(random_state=42)  # Set random state for reproducibility
linear_model.fit(X_train, Y_train)
print('Finish training LinearSVC ...')
total_time = getCurrentTimestamp() - start_time
print(total_time) #in seconds
y_pred = linear_model.predict(X_test) #predict by our model

Begin training LinearSVC, please wait ...
Finish training LinearSVC ...
818


In [68]:
accuracy = 100*metrics.accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 99.81481829885477


In [69]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    299577
           1       1.00      0.99      1.00     72490

    accuracy                           1.00    372067
   macro avg       1.00      1.00      1.00    372067
weighted avg       1.00      1.00      1.00    372067



In [ ]:
#Finish